## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-19-05-53-46 +0000,bbc,'Credit-score company encouraged me to borrow ...,https://www.bbc.com/news/articles/c4g0m1j4042o...
1,2026-01-19-05-50-36 +0000,nypost,12-year-old boy saved mom’s life after she los...,https://nypost.com/2026/01/19/world-news/12-ye...
2,2026-01-19-05-42-29 +0000,nypost,ICE agent slams protester during search for al...,https://nypost.com/2026/01/19/us-news/ice-agen...
3,2026-01-19-05-41-46 +0000,nyt,China’s Birthrate Plunges to Lowest Level Sinc...,https://www.nytimes.com/2026/01/18/business/ch...
4,2026-01-19-05-39-38 +0000,nyt,Trump’s Ultimatum to Europe,https://www.nytimes.com/2026/01/18/world/green...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2389/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
42,trump,47
267,greenland,21
23,ice,13
337,minnesota,9
51,least,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
207,2026-01-18-08-00-32 +0000,wapo,Melting ice may raise Greenland’s value. Trump...,https://www.washingtonpost.com/climate-environ...,96
138,2026-01-18-18-20-27 +0000,nypost,EU threatens trade ‘bazooka’ against US after ...,https://nypost.com/2026/01/18/world-news/europ...,95
147,2026-01-18-17-37-12 +0000,nypost,GOP lawmakers irked at Trump’s new planned ta...,https://nypost.com/2026/01/18/us-news/gopers-i...,94
215,2026-01-18-06-01-39 +0000,nyt,Trump Announces New Tariffs on Some European C...,https://www.nytimes.com/2026/01/17/us/politics...,92
210,2026-01-18-07-21-34 +0000,bbc,Trump tariff threat over Greenland 'unacceptab...,https://www.bbc.com/news/articles/cy4qjwk9n2no...,91


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
207,96,2026-01-18-08-00-32 +0000,wapo,Melting ice may raise Greenland’s value. Trump...,https://www.washingtonpost.com/climate-environ...
92,52,2026-01-18-23-14-31 +0000,cbc,At least 20 believed dead in high-speed train ...,https://www.cbc.ca/news/world/spain-train-cras...
88,38,2026-01-18-23-24-23 +0000,nypost,Pregnant mother of two ID’d among six women ki...,https://nypost.com/2026/01/18/world-news/pregn...
145,33,2026-01-18-17-50-47 +0000,nypost,Minneapolis Mayor Jacob Frey rips ICE’s immigr...,https://nypost.com/2026/01/18/us-news/minneapo...
50,31,2026-01-19-01-42-43 +0000,nypost,Brazen NYC carjacker swipes vehicle where blin...,https://nypost.com/2026/01/18/us-news/brazen-n...
132,30,2026-01-18-19-15-11 +0000,nypost,Stunned woman wakes up to find 8-foot python o...,https://nypost.com/2026/01/18/world-news/stunn...
163,29,2026-01-18-14-56-16 +0000,nypost,President Trump puts $1B price tag on permanen...,https://nypost.com/2026/01/18/us-news/presiden...
184,27,2026-01-18-11-00-00 +0000,latimes,Barabak: They were like oil and water. Then Ha...,https://www.latimes.com/politics/story/2026-01...
147,26,2026-01-18-17-37-12 +0000,nypost,GOP lawmakers irked at Trump’s new planned ta...,https://nypost.com/2026/01/18/us-news/gopers-i...
18,25,2026-01-19-04-54-00 +0000,nyt,European Union Officials Lean Toward Negotiati...,https://www.nytimes.com/2026/01/18/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
